In [29]:
import os

import pandas as pd
import tqdm

In [ ]:
df_sdg = pd.read_pickle("../data/dataframes/output/all_sdg_fixed_dst.pkl")
df_dt = pd.read_pickle("../data/dataframes/DT/dt_updated.pkl")
df_sdg = df_sdg[df_sdg.PT == 'J']
df_dt = df_dt[df_dt.PT == 'J']
df_inter = df_sdg[df_sdg['DST']]


In [13]:
eu = pd.read_excel("../data/countries_eu.xlsx", sheet_name='EU')['Country'].to_list()
eu

['Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden']

In [14]:
df_gpp_pop = pd.read_csv("../data/countriesgdp_pop.csv", index_col="Country").drop(['ranking', 'gdppc'], axis=1)


In [5]:
import json
with open('../data/iso3.json', 'r') as f:
    iso3 = json.load(f)
with open('../data/names.json', 'r') as f:
    names = json.load(f)

df_iso = pd.DataFrame.from_dict(data=iso3, orient="index", columns=['ISO3'])
df_name = pd.DataFrame.from_dict(data=names, orient="index", columns=['names'])
df_iso = pd.concat([df_iso, df_name], axis=1).reset_index().rename(columns={"index":"ISO2"}).set_index('ISO3')
df_iso.loc['EUU'] = ['EU', 'EU']
df_iso

,ISO2,names
ISO3,,
BGD,BD,Bangladesh
BEL,BE,Belgium
BFA,BF,Burkina Faso
BGR,BG,Bulgaria
BIH,BA,Bosnia and Herzegovina
...,...,...
IDN,ID,Indonesia
UKR,UA,Ukraine
QAT,QA,Qatar


In [6]:
pop = pd.read_excel("../data/pop.xlsx", index_col="Country Code")
pop = pop.loc[:, [i for i in range(2010,2022)]]
pop['avg_pop'] = pop.mean(axis=1)
pop = pop.merge(df_iso, left_index=True, right_index=True)
pop = pop.set_index('names').loc[:, "avg_pop"]

In [7]:
pop



names
Aruba           1.044919e+05
Afghanistan     3.468124e+07
Angola          2.846842e+07
Albania         2.875263e+06
Andorra         7.930583e+04
                    ...     
Kosovo          1.795317e+06
Yemen           2.682369e+07
South Africa    5.571583e+07
Zambia          1.616954e+07
Zimbabwe        1.389710e+07
Name: avg_pop, Length: 217, dtype: float64

In [8]:
gdp= pd.read_excel("../data/pop-gdp.xlsx", sheet_name="GDP", index_col="TIME")
gdp

,2020-Q2,Unnamed: 2,2020-Q3,Unnamed: 4,2020-Q4,Unnamed: 6,2021-Q1
TIME,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EU,3092983.9,NaN,3418271.6,NaN,3588456.7,NaN,3395193.7
European Union - 28 countries (2013-2020),:,NaN,:,NaN,:,NaN,:
Euro area - 19 countries (from 2015),2631060.9,NaN,2909608.3,NaN,3035281.1,NaN,2902475.8
Belgium,105644,p,113945,p,124206,p,118204
...,...,...,...,...,...,...,...
Palestine,:,NaN,:,NaN,:,NaN,:
Saudi Arabia,:,NaN,:,NaN,:,NaN,:
Australia,280349.7,NaN,296909.3,NaN,323249,NaN,324962.5


In [9]:
gdppop = pd.merge(left=pop, right=gdp.loc[:, "2021-Q1"], left_index=True, right_index=True)
gdppop = gdppop.rename(columns={"2021-Q1":"gdp"})
gdppop.to_excel("../data/gdp_avgpop.xlsx")

In [15]:
gdppop = pd.read_excel("../data/gdp_avgpop.xlsx", index_col=0)
df_c_wos = pd.read_excel("/home/kevin-desktop/PycharmProjects/SDG/data/total_number_pubs.xlsx", index_col=0)


In [16]:
tot = df_c_wos.transpose().rename(columns={'Total from Wos':'tot'}).loc[:, "tot"]
tot

World             49621592
EU                13486423
China              6297302
United States     14435821
Austria             405283
Belgium           590783.0
Bulgaria             97530
Croatia            94711.0
Hungary           221415.0
Cyprus               30929
Czech Republic      343569
Denmark             481528
Estonia              46169
Finland             404535
France             2136752
Germany            2821820
Greece              322888
Ireland             297273
Italy              1877833
Latvia               30904
Lithuania            58961
Luxembourg           22711
Malta                 8394
Netherlands        1106335
Poland              734001
Portugal            327991
Romania             253363
Slovakia            114908
Slovenia             93129
Spain              1354702
Sweden              824499
Iceland              25244
Liechtenstein         1891
Norway              366460
Switzerland         784015
United Kingdom     3878133
Japan              3052441
S

In [17]:
calc = pd.merge(tot, gdppop, left_index=True, right_index=True)
calc

,tot,avg_pop,gdp
EU,13486423,4.445399e+08,3395193.7
China,6297302,1.380822e+09,3176037.4
United States,14435821,3.213942e+08,4518929.3
Austria,405283,8.665144e+06,92140.6
Belgium,590783.0,1.128651e+07,118204
Bulgaria,97530,7.146214e+06,14512.3
Croatia,94711.0,4.161662e+06,12658.3
Hungary,221415.0,9.841952e+06,31845.5
Cyprus,30929,1.165859e+06,5570.4
Czech Republic,343569,1.057757e+07,53083.1


In [18]:
from collections import Counter
dic_countries = {country:Counter() for country in calc.index}
dic_countries_frac = {country:Counter() for country in calc.index}

In [19]:
idx_sdg = {name: i for i, name in enumerate(list(df_sdg), start=0)}
idx_dt = {name: i for i, name in enumerate(list(df_dt), start=0)}
idx_sdg_dt = {name: i for i, name in enumerate(list(df_inter), start=0)}
idx_sdg_dt

{'PT': 0,
 'AU': 1,
 'TI': 2,
 'SO': 3,
 'DE': 4,
 'AB': 5,
 'C1': 6,
 'EM': 7,
 'TC': 8,
 'PY': 9,
 'WC': 10,
 'UT': 11,
 'CN': 12,
 'SDG1': 13,
 'SDG2': 14,
 'SDG3': 15,
 'SDG4': 16,
 'SDG5': 17,
 'SDG6': 18,
 'SDG7': 19,
 'SDG8': 20,
 'SDG9': 21,
 'SDG10': 22,
 'SDG11': 23,
 'SDG12': 24,
 'SDG13': 25,
 'SDG14': 26,
 'SDG15': 27,
 'SDG16': 28,
 'SDG17': 29,
 'AI': 30,
 'big_data': 31,
 'IOT': 32,
 'computing_infrastructure': 33,
 'blockchain': 34,
 'robotics': 35,
 'additive_manufacturing': 36,
 'Society': 37,
 'Economy': 38,
 'Environment': 39,
 'EU': 40,
 'DST': 41}

In [20]:
# Digital
for row in df_dt.itertuples(index=False, name=None):
    cn = row[idx_dt['CN']]
    lst_actors = cn.split(", ")
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries.keys():
            # Increment for digital
            dic_countries_frac[actor]['dt'] += 1/size_actors
            dic_countries[actor]['dt'] += 1
            if actor in eu:
                dic_countries_frac['EU']['dt'] += 1/size_actors
                dic_countries['EU']['dt'] += 1
# SDG & SDG-DT
for row in df_sdg.itertuples(index=False, name=None):
    cn = row[idx_sdg['CN']]
    lst_actors = cn.split(", ")
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries.keys():
            # Increment for SDG
            dic_countries_frac[actor]['sdg'] += 1/size_actors
            dic_countries[actor]['sdg'] += 1
            if actor in eu:
                dic_countries_frac['EU']['sdg'] += 1/size_actors
                dic_countries['EU']['sdg'] += 1
            if row[-1]:
                # Increment for SDG-DT
                dic_countries_frac[actor]['inter'] += 1/size_actors
                dic_countries[actor]['inter'] += 1
                if actor in eu:
                    dic_countries_frac['EU']['inter'] += 1/size_actors
                    dic_countries['EU']['inter'] += 1

In [21]:
df_full = pd.DataFrame.from_dict(dic_countries, orient='index')
df_full.rename(columns={'dt':'dt_full', 'sdg':'sdg_full', 'inter':'inter_full'}, inplace=True)
df_full

,dt_full,sdg_full,inter_full
EU,780275,1287481,35322
China,928484,827602,34202
United States,575142,871274,24155
Austria,16390,27193,753
Belgium,27993,45638,1045
Bulgaria,2246,4050,96
Croatia,4879,11505,318
Hungary,7671,13404,396
Cyprus,2237,3641,181
Czech Republic,13016,23147,597


In [22]:
df_frac = pd.DataFrame.from_dict(data=dic_countries_frac, orient='index')
df_frac.rename(columns={'dt':'dt_frac', 'sdg':'sdg_frac', 'inter':'inter_frac'}, inplace=True)
df_frac

,dt_frac,sdg_frac,inter_frac
EU,149626.572842,255032.298970,6973.571454
China,188528.693224,145199.568982,6755.196249
United States,114589.956220,193487.922934,4689.826355
Austria,3236.053303,5372.635315,138.258313
Belgium,4779.746296,8371.507113,181.901241
Bulgaria,549.483471,1009.169897,24.554932
Croatia,1225.321102,2621.667351,78.026927
Hungary,1670.516461,2767.537751,85.481166
Cyprus,541.687904,851.738882,41.854211
Czech Republic,2647.521995,4618.636023,107.792072


In [23]:
# final = calc.merge(df_full, left_index=True, right_index=True).merge(df_frac, left_index=True, right_index=True)
final = calc.merge(df_frac, left_index=True, right_index=True)

In [24]:
final = final.round(2)
final

,tot,avg_pop,gdp,dt_frac,sdg_frac,inter_frac
EU,13486423,4.445399e+08,3395193.7,149626.57,255032.30,6973.57
China,6297302,1.380822e+09,3176037.4,188528.69,145199.57,6755.20
United States,14435821,3.213942e+08,4518929.3,114589.96,193487.92,4689.83
Austria,405283,8.665144e+06,92140.6,3236.05,5372.64,138.26
Belgium,590783.0,1.128651e+07,118204,4779.75,8371.51,181.90
Bulgaria,97530,7.146214e+06,14512.3,549.48,1009.17,24.55
Croatia,94711.0,4.161662e+06,12658.3,1225.32,2621.67,78.03
Hungary,221415.0,9.841952e+06,31845.5,1670.52,2767.54,85.48
Cyprus,30929,1.165859e+06,5570.4,541.69,851.74,41.85
Czech Republic,343569,1.057757e+07,53083.1,2647.52,4618.64,107.79


In [25]:
final_gdp = final.loc[:, ['tot', 'gdp', 'dt_frac', 'sdg_frac', 'inter_frac']]
final_gdp.loc[:, 'dt_gdp'] = final_gdp.loc[:, 'dt_frac'] / final_gdp.loc[:, 'gdp']
final_gdp.loc[:, 'sdg_gdp'] = final_gdp.loc[:, 'sdg_frac'] / final_gdp.loc[:, 'gdp']
final_gdp.loc[:, 'inter_gdp'] = final_gdp.loc[:, 'inter_frac'] / final_gdp.loc[:, 'gdp']
final_gdp = final_gdp.round(2)
final_gdp

,tot,gdp,dt_frac,sdg_frac,inter_frac,dt_gdp,sdg_gdp,inter_gdp
EU,13486423,3395193.7,149626.57,255032.30,6973.57,0.04407,0.075116,0.002054
China,6297302,3176037.4,188528.69,145199.57,6755.20,0.05936,0.045717,0.002127
United States,14435821,4518929.3,114589.96,193487.92,4689.83,0.025358,0.042817,0.001038
Austria,405283,92140.6,3236.05,5372.64,138.26,0.035121,0.058309,0.001501
Belgium,590783.0,118204,4779.75,8371.51,181.90,0.040436,0.070823,0.001539
Bulgaria,97530,14512.3,549.48,1009.17,24.55,0.037863,0.069539,0.001692
Croatia,94711.0,12658.3,1225.32,2621.67,78.03,0.0968,0.207111,0.006164
Hungary,221415.0,31845.5,1670.52,2767.54,85.48,0.052457,0.086905,0.002684
Cyprus,30929,5570.4,541.69,851.74,41.85,0.097244,0.152905,0.007513
Czech Republic,343569,53083.1,2647.52,4618.64,107.79,0.049875,0.087008,0.002031


In [26]:
final_pop = final.loc[:, ['tot', 'avg_pop', 'dt_frac', 'sdg_frac', 'inter_frac']]
final_pop.loc[:, 'dt_pop'] = 10*final_pop.loc[:, 'dt_frac'] / final_pop.loc[:, 'avg_pop']
final_pop.loc[:, 'sdg_pop'] = 10*final_pop.loc[:, 'sdg_frac'] / final_pop.loc[:, 'avg_pop']
final_pop.loc[:, 'inter_pop'] = 100 * final_pop.loc[:, 'inter_frac'] / final_pop.loc[:, 'avg_pop']
final_pop = final_pop.round(2)
final_pop

,tot,avg_pop,dt_frac,sdg_frac,inter_frac,dt_pop,sdg_pop,inter_pop
EU,13486423,4.445399e+08,149626.57,255032.30,6973.57,0.00,0.01,0.0
China,6297302,1.380822e+09,188528.69,145199.57,6755.20,0.00,0.00,0.0
United States,14435821,3.213942e+08,114589.96,193487.92,4689.83,0.00,0.01,0.0
Austria,405283,8.665144e+06,3236.05,5372.64,138.26,0.00,0.01,0.0
Belgium,590783.0,1.128651e+07,4779.75,8371.51,181.90,0.00,0.01,0.0
Bulgaria,97530,7.146214e+06,549.48,1009.17,24.55,0.00,0.00,0.0
Croatia,94711.0,4.161662e+06,1225.32,2621.67,78.03,0.00,0.01,0.0
Hungary,221415.0,9.841952e+06,1670.52,2767.54,85.48,0.00,0.00,0.0
Cyprus,30929,1.165859e+06,541.69,851.74,41.85,0.00,0.01,0.0
Czech Republic,343569,1.057757e+07,2647.52,4618.64,107.79,0.00,0.00,0.0


In [27]:

with pd.ExcelWriter('recap_giacomo.xlsx') as writer:
    final.to_excel(writer, sheet_name='raw')
    final_gdp.to_excel(writer, sheet_name='gdp')
    final_pop.to_excel(writer, sheet_name='pop')

In [90]:
df_dt

,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,...,UT,CN,AI,big_data,IOT,computing_infrastructure,blockchain,robotics,additive_manufacturing,EU
0,J,"Ondra, V; Sever, IA; Schwingshackl, CW",A method for detection and characterisation of...,MECHANICAL SYSTEMS AND SIGNAL PROCESSING,Non-linear system characterisation; Hilbert tr...,This paper presents a method for detection and...,"[Ondra, V.; Schwingshackl, C. W.] Imperial Col...",v.ondra14@imperial.ac.uk; Ibrahim.Sever@Rolls-...,26,2017,...,WOS:000385209000012,"United Kingdom, United Kingdom, United Kingdom",True,False,False,False,False,False,False,True
1,J,"Liu, K; Kang, G",Multiview convolutional neural networks for lu...,INTERNATIONAL JOURNAL OF IMAGING SYSTEMS AND T...,lung nodule classification; multiview convolut...,To find a better way to screen early lung canc...,"[Liu, Kui; Kang, Guixia] Beijing Univ Posts & ...",liukui_006@126.com,55,2017,...,WOS:000397734700002,"China, China, China",True,False,False,False,False,False,False,False
2,J,"Ali, MS; Balasubramaniam, P; Zhu, QX",Stability of stochastic fuzzy BAM neural netwo...,INTERNATIONAL JOURNAL OF MACHINE LEARNING AND ...,Discrete and distributed time-varying delay; G...,"Among the various fuzzy models, the well-known...","[Ali, M. Syed] Thiruvalluvar Univ, Dept Math, ...",syedgru@gmail.com; zqx22@126.com,39,2017,...,WOS:000393309600022,"India, India, China",True,False,False,False,False,False,False,False
3,J,"Benvidi, A; Abbasi, S; Gharaghani, S; Tezerjan...",Spectrophotometric determination of synthetic ...,FOOD CHEMISTRY,Multicomponent analysis; Artificial neural net...,"Four common food colorants, containing tartraz...","[Benvidi, Ali; Abbasi, Saleheh; Tezerjani, Mar...",abenvidi@yazd.ac.ir,48,2017,...,WOS:000388112500048,,True,False,False,False,False,False,False,False
4,J,"Rajagopal, K; Balakrishnan, SN; Busemeyer, JR",Neural Network-Based Solutions for Stochastic ...,IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARN...,Adaptive dynamic programming; neural network-b...,"In this paper, an offline approximate dynamic ...","[Rajagopal, Karthikeyan] Gen Motors Co, Global...",krg2d@mst.edu; bala@mst.edu; jbusemey@indiana.edu,12,2017,...,WOS:000395980500005,"United States, United States, United States, I...",True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253123,J,"Wei, FF; Feng, JQ",Real-time ray casting of algebraic B-spline su...,COMPUTERS & GRAPHICS-UK,Piecewise algebraic surface; B-spline; Newton-...,Piecewise algebraic B-spline surfaces (ABS sur...,"[Wei, Feifei; Feng, Jieqing] Zhejiang Univ, St...",weifeifeicad@gmail.com; jqfeng@cad.zju.edu.cn,2,2011,...,WOS:000294034600008,"China, China",False,False,False,True,False,False,False,False
1253124,C,"Hu, Y; Huang, XM; Baker, AH; Tseng, YH; Bryan,...",Improving the Scalability of the Ocean Barotro...,PROCEEDINGS OF SC15: THE INTERNATIONAL CONFERE...,parallel computing; linear solver; ocean modeling,High-resolution climate simulations are increa...,"[Hu, Yong; Huang, Xiaomeng; Yang, Guangwen] Ts...",huyong11@tsinghua.edu.cn; hxm@tsinghua.edu.cn;...,10,2015,...,WOS:000382162500043,"China, China, China, United States, United Sta...",False,False,False,True,False,False,False,False
1253125,J,"Kim, TJ; Sun, X; Yoon, SE",T-ReX: Interactive Global Illumination of Mass...,IEEE TRANSACTIONS ON VISUALIZATION AND COMPUTE...,Massive models; ray tracing; photon mapping; g...,We propose several interactive global illumina...,"[Kim, Tae-Joon; Yoon, Sung-Eui] Korea Adv Inst...",tjkim.kaist@gmail.com; sunxin@microsoft.com; s...,6,2014,...,WOS:000338118700013,"South Korea, South Korea, China",False,False,False,True,False,False,False,False
1253126,J,"Kavouklis, C; Kallinderis, Y",Parallel adaptation of general three-dimension...,JOURNAL OF COMPUTATIONAL PHYSICS,Parallel computing; Hybrid meshes; Mesh adapta...,A new parallel dynamic mesh adaptation and loa...,"[Kavouklis, Christos; Kallinderis, Yannis] Uni...",christos.kavou

In [91]:
df_dt.groupby('AI').count()

,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,WC,UT,CN,big_data,IOT,computing_infrastructure,blockchain,robotics,additive_manufacturing,EU
AI,,,,,,,,,,,,,,,,,,,,
False,532551,532551,532551,532551,532551,532551,532551,499246,532551,532551,532320,532551,532551,532551,532551,532551,532551,532551,532551,532551
True,720577,720577,720577,720577,720577,720577,720577,670621,720577,720577,720404,720577,720577,720577,720577,720577,720577,720577,720577,720577


In [105]:
round(720577 / (720577+532551),3)

0.575

In [97]:
(720577+532551)

1253128

In [92]:
df_dt.groupby('robotics').count()

,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,WC,UT,CN,AI,big_data,IOT,computing_infrastructure,blockchain,additive_manufacturing,EU
robotics,,,,,,,,,,,,,,,,,,,,
False,1044609,1044609,1044609,1044609,1044609,1044609,1044609,972496,1044609,1044609,1044277,1044609,1044609,1044609,1044609,1044609,1044609,1044609,1044609,1044609
True,208519,208519,208519,208519,208519,208519,208519,197371,208519,208519,208447,208519,208519,208519,208519,208519,208519,208519,208519,208519


In [104]:
round(208519/1253128, 3)

0.166

In [93]:
df_dt.groupby('IOT').count()

,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,WC,UT,CN,AI,big_data,computing_infrastructure,blockchain,robotics,additive_manufacturing,EU
IOT,,,,,,,,,,,,,,,,,,,,
False,1081659,1081659,1081659,1081659,1081659,1081659,1081659,1009802,1081659,1081659,1081321,1081659,1081659,1081659,1081659,1081659,1081659,1081659,1081659,1081659
True,171469,171469,171469,171469,171469,171469,171469,160065,171469,171469,171403,171469,171469,171469,171469,171469,171469,171469,171469,171469


In [103]:
round(171469/1253128,3
      )

0.137

In [94]:
df_dt.groupby('blockchain').count()

,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,WC,UT,CN,AI,big_data,IOT,computing_infrastructure,robotics,additive_manufacturing,EU
blockchain,,,,,,,,,,,,,,,,,,,,
False,1237321,1237321,1237321,1237321,1237321,1237321,1237321,1155047,1237321,1237321,1236994,1237321,1237321,1237321,1237321,1237321,1237321,1237321,1237321,1237321
True,15807,15807,15807,15807,15807,15807,15807,14820,15807,15807,15730,15807,15807,15807,15807,15807,15807,15807,15807,15807


In [102]:
round(15807/1253128, 3)

0.013

In [110]:
df_dt = pd.read_pickle("../data/dataframes/DT/digital_1975-2022.pkl")
df_iot = df_dt[df_dt['IOT']]

In [111]:
df_iot.groupby('PY')['PT'].count()

PY
1991        6
1992        9
1993       13
1994        9
1995       10
1996       14
1997       25
1998      124
1999      124
2000      160
2001      187
2002      305
2003      431
2004      750
2005     1224
2006     1827
2007     2961
2008     3545
2009     4868
2010     4196
2011     5066
2012     6017
2013     7688
2014     9352
2015    12999
2016    16410
2017    20093
2018    24112
2019    27783
2020    27688
2021    10027
Name: PT, dtype: int64

In [125]:
## Check IOT
import os
import csv
import tqdm

In [129]:
root = "../data/raw_data/IOT/"
lst_df = []
for file in tqdm.tqdm(os.listdir(root)):
    path = os.path.join(root, file)
    df = pd.read_csv(path, sep='\t', encoding='utf-8',
                                 index_col=False, on_bad_lines="skip",
                                 quoting=csv.QUOTE_NONE)
    df = df[['UT','AB', 'TI', 'PY', 'LA']]
    df.dropna(subset='UT')
    lst_df.append(df)

100%|██████████| 506/506 [00:08<00:00, 60.48it/s]


In [147]:
df_iot = pd.concat(lst_df)
df_iot.drop_duplicates(subset="UT", keep="first", inplace=True)
df_iot.drop_duplicates(subset="AB", keep="first", inplace=True)
df_iot.drop_duplicates(subset="TI", keep="first", inplace=True)

df_iot.dropna(inplace=True)
df_iot = df_iot[df_iot['LA'] == "English"].drop(columns=['LA'])
df_iot

,UT,AB,TI,PY
0,WOS:000556596600252,Blockchain has been hailed as an emerging tech...,Avoiding Risky Designs When Using Blockchain T...,2019.0
1,WOS:000533346400271,Cyber-physical systems (CPSs) have sophisticat...,Implementation of System Operation Modes for H...,2020.0
2,WOS:000483812900001,Cybersecurity for the production of safe and e...,Cyberbiosecurity in Advanced Manufacturing Models,2019.0
3,WOS:000539640300011,The congestion of urban car traffic creates ma...,Urban Intersection Management with Connected I...,2019.0
4,WOS:000600287000004,Computer memory occupation is an important iss...,Adaptive Secure State Estimation for Cyber-Phy...,2020.0
...,...,...,...,...
492,WOS:000594337700076,This article presents a mixed signal-based nov...,Design Principles for a Novel Lightweight Conf...,2020.0
493,WOS:000491295800135,Due to the depletion of energy resources and i...,Design and Development of an IoT Gateway for S...,2019.0
495,WOS:000587291400061,Human safety is an important issue due to the ...,Human Tracking System Based on GPS and IOT (In...,2020.0
496,WOS:000529962800012,E-health systems can be used to monitor people...,The internet of things for healthcare: optimis...,2020.0


In [149]:
df_iot['PY'] = df_iot['PY'].astype("int")
df_iot['PY'] = df_iot['PY'].astype("category")
df_iot

,UT,AB,TI,PY
0,WOS:000556596600252,Blockchain has been hailed as an emerging tech...,Avoiding Risky Designs When Using Blockchain T...,2019
1,WOS:000533346400271,Cyber-physical systems (CPSs) have sophisticat...,Implementation of System Operation Modes for H...,2020
2,WOS:000483812900001,Cybersecurity for the production of safe and e...,Cyberbiosecurity in Advanced Manufacturing Models,2019
3,WOS:000539640300011,The congestion of urban car traffic creates ma...,Urban Intersection Management with Connected I...,2019
4,WOS:000600287000004,Computer memory occupation is an important iss...,Adaptive Secure State Estimation for Cyber-Phy...,2020
...,...,...,...,...
492,WOS:000594337700076,This article presents a mixed signal-based nov...,Design Principles for a Novel Lightweight Conf...,2020
493,WOS:000491295800135,Due to the depletion of energy resources and i...,Design and Development of an IoT Gateway for S...,2019
495,WOS:000587291400061,Human safety is an important issue due to the ...,Human Tracking System Based on GPS and IOT (In...,2020
496,WOS:000529962800012,E-health systems can be used to monitor people...,The internet of things for healthcare: optimis...,2020


In [150]:
df_iot.groupby('PY')['UT'].count()

PY
2010     5817
2011     6635
2012     7974
2013    10211
2014    12216
2015    16732
2016    20534
2017    24935
2018    28983
2019    32135
2020    31445
2021    11255
2022      165
Name: UT, dtype: int64

In [152]:
lst_iot = pd.read_csv("../data/keyword_digital_simplified.csv", sep='\t').loc[:, "IOT"].dropna().to_list()
lst_iot = [term.replace("*", r"\w*") for term in lst_iot]
lst_iot

['Connected device\\w*',
 'Connected home\\w*',
 'Cyber-physical system\\w*',
 'Human-machine interface\\w*',
 'Hyper connectivity',
 'Industrial internet of things',
 'Intelligent factor\\w*',
 'Internet of everything',
 'Internet of things',
 'IoT',
 'Machine-to-enterprise',
 'Machine-to-human',
 'Machine-to-machine',
 'Pervasive sensing',
 'Sensor network\\w*',
 'Smart device\\w*',
 'Smart factor\\w*',
 'Smart home\\w*',
 'Smart sensor\\w*',
 'Wearable\\w*',
 'Wireless body area network\\w*',
 'Wireless sensor network\\w*']

In [153]:
import re
dic_reg = {term: re.compile(term, flags=re.IGNORECASE) for term in lst_iot}

In [156]:
dic_iot_terms = {term:Counter() for term in lst_iot}
for row in tqdm.tqdm(df_iot.itertuples(index=False, name=None), total=df_iot.shape[0]):
    ab = row[1]
    py = row[3]
    for term in lst_iot:
        m = re.search(dic_reg[term], string=ab)
        if m:
            dic_iot_terms[term][int(py)] += 1





100%|██████████| 209037/209037 [00:44<00:00, 4720.26it/s]


In [193]:
df_terms = pd.DataFrame.from_dict(dic_iot_terms, orient='index')

In [194]:
df_terms = df_terms.loc[:, [i for i in range(2010,2023)]]

In [209]:
df_t = df_terms.transpose().loc[2010:2021, :].fillna(0)
df_t.loc['tot', :] = df_t.sum()
mask = df_t.loc['tot', :] > 10000

df_plot = df_t.loc[2010:2021, mask]
df_plot

,Internet of things,IoT,Sensor network\w*,Wearable\w*,Wireless sensor network\w*
2010,85.0,74.0,3924.0,502.0,2540.0
2011,290.0,180.0,4017.0,581.0,2801.0
2012,472.0,320.0,4574.0,669.0,3229.0
2013,761.0,512.0,5533.0,1023.0,3916.0
2014,1259.0,972.0,5760.0,1525.0,4200.0
2015,2394.0,2135.0,6673.0,2669.0,4886.0
2016,3946.0,3883.0,6338.0,3788.0,4651.0
2017,5859.0,6187.0,6117.0,4870.0,4428.0
2018,7793.0,8986.0,5548.0,5670.0,3953.0
2019,9310.0,10921.0,5045.0,6753.0,3522.0


In [210]:
import plotly.express as px
fig = px.line(data_frame=df_plot)
fig.update_layout(template="simple_white")

In [181]:
dic_countries_frac = {country:{year:0 for year in range(2010,2022)} for country in final_pop.index}

In [182]:
dic_countries_frac

{'EU': {2010: 0,
  2011: 0,
  2012: 0,
  2013: 0,
  2014: 0,
  2015: 0,
  2016: 0,
  2017: 0,
  2018: 0,
  2019: 0,
  2020: 0,
  2021: 0},
 'China': {2010: 0,
  2011: 0,
  2012: 0,
  2013: 0,
  2014: 0,
  2015: 0,
  2016: 0,
  2017: 0,
  2018: 0,
  2019: 0,
  2020: 0,
  2021: 0},
 'United States': {2010: 0,
  2011: 0,
  2012: 0,
  2013: 0,
  2014: 0,
  2015: 0,
  2016: 0,
  2017: 0,
  2018: 0,
  2019: 0,
  2020: 0,
  2021: 0},
 'Austria': {2010: 0,
  2011: 0,
  2012: 0,
  2013: 0,
  2014: 0,
  2015: 0,
  2016: 0,
  2017: 0,
  2018: 0,
  2019: 0,
  2020: 0,
  2021: 0},
 'Belgium': {2010: 0,
  2011: 0,
  2012: 0,
  2013: 0,
  2014: 0,
  2015: 0,
  2016: 0,
  2017: 0,
  2018: 0,
  2019: 0,
  2020: 0,
  2021: 0},
 'Bulgaria': {2010: 0,
  2011: 0,
  2012: 0,
  2013: 0,
  2014: 0,
  2015: 0,
  2016: 0,
  2017: 0,
  2018: 0,
  2019: 0,
  2020: 0,
  2021: 0},
 'Croatia': {2010: 0,
  2011: 0,
  2012: 0,
  2013: 0,
  2014: 0,
  2015: 0,
  2016: 0,
  2017: 0,
  2018: 0,
  2019: 0,
  2020: 0,
  20

In [183]:
for row in df_sdg.itertuples(index=False, name=None):
    cn = row[idx_sdg['CN']]
    year = row[idx_sdg['PY']]
    lst_actors = cn.split(", ")
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries.keys():
            if row[-1]:
                # Increment for SDG-DT
                dic_countries_frac[actor][year] += 1/size_actors
                if actor in eu:
                    dic_countries_frac['EU'][year] += 1/size_actors

In [190]:
df = pd.DataFrame.from_dict(dic_countries_frac, orient="index")
df['Total'] = df.sum(axis=1)

In [191]:
df.loc[['China', 'United States', 'EU']]

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total
China,160.785281,178.994841,234.190079,288.192309,337.451587,431.823218,592.340832,779.611348,992.391906,1386.553594,1741.816514,2265.974979,9390.126490
United States,135.587987,163.519653,200.031770,259.282901,297.032873,351.514630,402.445601,560.277000,678.863487,893.894368,1162.200222,1406.312465,6510.962957
EU,174.462121,205.380402,247.216389,328.314121,406.139159,565.115220,697.924820,841.431715,1143.185875,1359.312472,1882.816677,2367.075302,10218.374272
